# 05 Backtesting

Simulate portfolio performance using predicted residuals.


In [ ]:
from src.data_utils import load_pickle
from src.portfolio import run_long_short_strategy
from src.evaluation import evaluate_performance, plot_performance

import pandas as pd

preds = load_pickle('../data/processed/predictions.pkl')
# Simulate returns for demo
returns = preds.copy()
returns[:] = 0.001  # dummy constant returns
portfolio_returns = run_long_short_strategy(preds, returns)
metrics = evaluate_performance(portfolio_returns)
print(metrics)
plot_performance(portfolio_returns)


In [ ]:
# 05 Backtesting

from src.data_utils import load_pickle
from src.portfolio import run_long_short_strategy
from src.evaluation import evaluate_performance, plot_performance

import pandas as pd
import numpy as np

# Load predictions and test-period actual returns
preds_df = load_pickle('../data/processed/predictions.pkl')
# preds_df has columns: ['preds', 'y_test'] and y_test index

# For backtest, assume one "rebalance period" per row (one per date-ticker combo)
# In a real workflow, you would use a DataFrame with [dates, tickers] as index and run cross-sectional portfolio formation

# For demo, aggregate by date
if isinstance(preds_df.index, pd.MultiIndex):
    preds_by_date = preds_df.reset_index().groupby('date').agg({'preds':'mean', 'y_test':'mean'})
else:
    preds_by_date = preds_df

# Simulate returns: if you have true returns, use them! Here we'll use y_test as realized return.
# For each period: "go long" when prediction is high, "short" when prediction is low (simple directional bet)
returns = preds_by_date['y_test']
signals = preds_by_date['preds']

# Long-short: long if prediction > 0, short if < 0
portfolio_returns = []
for s, r in zip(signals, returns):
    if s > 0:
        portfolio_returns.append(r)
    else:
        portfolio_returns.append(-r)
portfolio_returns = pd.Series(portfolio_returns, index=preds_by_date.index)

# Evaluate and plot
metrics = evaluate_performance(portfolio_returns)
print("Backtest Performance Metrics:", metrics)
plot_performance(portfolio_returns)
